https://adventofcode.com/2023/day/18

In [1]:
from collections import deque
from itertools import pairwise

import networkx as nx

In [2]:
with open("data/18.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
"""

In [4]:
def parse_puzzle(puzzle):
    G = nx.Graph()
    a = 0j
    for line in puzzle.splitlines():
        drxn_str, steps_str, color_str = line.split()
        drxn = {"R": 1 + 0j, "U": 0 + 1j, "L": -1 + 0j, "D": 0 - 1j}[drxn_str]
        steps = int(steps_str)
        color = int(color_str[2:-1], base=16)
        for _ in range(steps):
            b = a + drxn
            G.add_node(b, color=color)
            G.add_edge(a, b)
            a = b
    return G

In [5]:
def find_interior_point(G):
    (xmin, xmax), (ymin, ymax) = find_extrema(G)

    def findit():
        for y in range(ymin + 1, ymax):
            for x in range(xmin, xmax):
                if x + y * 1j in G:
                    if x + 1 + y * 1j in G:
                        continue
                    else:
                        return x + 1 + y * 1j
        raise ValueError("No interiot points found")

    return findit()


def find_extrema(G):
    xmax = ymax = -float("inf")
    xmin = ymin = float("inf")
    for node in G:
        x, y = int(node.real), int(node.imag)
        xmin = min(x, xmin)
        xmax = max(x, xmax)
        ymin = min(y, ymin)
        ymax = max(y, ymax)
    return (xmin, xmax), (ymin, ymax)


def cubic_meters(puzzle):
    G = parse_puzzle(puzzle)
    p = find_interior_point(G)
    q = deque([p])
    visited = set()
    (xmin, xmax), (ymin, ymax) = find_extrema(G)
    while q:
        p = q.popleft()
        x, y = int(p.real), int(p.imag)
        if not xmin <= x <= xmax and ymin <= y <= ymax:
            raise ValueError("Escaped! : %s" % p)
        if p in visited:
            continue
        visited.add(p)
        for d in [1, 1j, -1, -1j]:
            nabe = p + d
            if nabe not in visited and nabe not in G:
                q.append(nabe)
    return len(visited) + len(G)

In [6]:
cubic_meters(testdata)

62

In [7]:
%%time
cubic_meters(data)

CPU times: user 50 ms, sys: 0 ns, total: 50 ms
Wall time: 49.7 ms


48652

### Part 2
Have to rethink

In [8]:
def shoelace(vertices):
    return shoelace_pairs(pairwise(vertices))


def shoelace_pairs(vertex_pairs):
    return sum((x1 * y2 - y1 * x2) for ((x1, y1), (x2, y2)) in vertex_pairs) / 2


def c2xy(c):
    return c.real, c.imag


def parse_puzzle_1(puzzle):
    G = nx.Graph()
    a = 0j
    for line in puzzle.splitlines():
        drxn_str, steps_str, color_str = line.split()
        drxn = {"R": 1 + 0j, "U": 0 + 1j, "L": -1 + 0j, "D": 0 - 1j}[drxn_str]
        steps = int(steps_str)
        b = a + steps * drxn
        G.add_node(a, dout=drxn, steps=steps)
        G.add_node(b, din=drxn)
        G.add_edge(a, b)
        a = b
    return G


def parse_puzzle_2(puzzle):
    G = nx.Graph()
    a = 0j
    for line in puzzle.splitlines():
        colorcode = line[-7:-1]
        steps = int(colorcode[:5], base=16)
        drxn = {"0": 1 + 0j, "3": 0 + 1j, "2": -1 + 0j, "1": 0 - 1j}[colorcode[5]]
        b = a + steps * drxn
        G.add_node(a, dout=drxn, steps=steps)
        G.add_node(b, din=drxn)
        G.add_edge(a, b)
        a = b
    return G


def cubic_meters_the_right_way(G):
    turn_corxns = {(-1j, -1j): 1, (-1j, 1j): 0, (1j, 1j): -1, (1j, -1j): 0}
    point = 0j
    outer_vertices = [point]
    turn_total = sum(d["dout"] / d["din"] for (n, d) in G.nodes(data=True))
    if turn_total == -4j:
        clockwise = 1
    elif turn_total == 4j:
        clockwise = -1
    else:
        raise ValueError("Invalid turn total: {}".format(turn_total))
    for a, b in nx.find_cycle(G, point):
        a = G.nodes[a]
        b = G.nodes[b]
        aturn = a["dout"] / a["din"] * clockwise
        bturn = b["dout"] / b["din"] * clockwise
        corxn = turn_corxns.get((aturn, bturn), 0)
        drxn, steps = a["dout"], a["steps"] + corxn
        point += drxn * steps
        outer_vertices.append(point)

    return shoelace(c2xy(c) for c in outer_vertices) * -clockwise

In [9]:
cubic_meters_the_right_way(parse_puzzle_1(testdata))

62.0

In [10]:
cubic_meters_the_right_way(parse_puzzle_1(data))

48652.0

In [11]:
%%time
cubic_meters_the_right_way(parse_puzzle_2(testdata))

CPU times: user 161 µs, sys: 0 ns, total: 161 µs
Wall time: 163 µs


952408144115.0

In [12]:
%%time
cubic_meters_the_right_way(parse_puzzle_2(data))

CPU times: user 6.48 ms, sys: 0 ns, total: 6.48 ms
Wall time: 6.46 ms


45757884535661.0